# FTEC5660 Homework 2 Part 2 
STUDENT ID: 1155254873


# Setup your agent

In [2]:
# Load environment variables
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
# Initialize LLM using ARK API
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name=os.getenv("ARK_API_MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("ARK_API_URL"),
    temperature=0,
)

# Create a moltbook account for your agent

In [25]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [26]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
student_id = int(1155254873)
encode_student_id(student_id)

'69918512'

In [1]:
import requests
response = requests.post(
    "https://www.moltbook.com/api/v1/agents/register",
    headers={"Content-Type": "application/json"},
    json={
        "name": "69918512",
        "description": "HW2p2"
    }
)
result = response.json()
result

{'success': False,
 'error': 'Agent name already taken',
 'hint': 'The name "69918512" is already registered. Try a different name.',
 'can_retry': True}

# Create a complete tool set to interact with moltbook

In [ ]:
import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = os.getenv('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- GET SUBMOLTS ----------
@tool
def get_submolts(limit: int = 20) -> dict:
    """Get list of available submolts (communities)."""
    r = requests.get(
        f"{BASE_URL}/submolts",
        headers=HEADERS,
        params={"limit": limit},
        timeout=15
    )
    return r.json()

# ---------- GET POSTS ----------
@tool
def get_posts(submolt: str = None, sort: str = "hot", limit: int = 10) -> dict:
    """Get posts, optionally filtered by submolt."""
    params = {"sort": sort, "limit": limit}
    if submolt:
        params["submolt"] = submolt
    r = requests.get(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        params=params,
        timeout=15
    )
    return r.json()

# ---------- SUBSCRIBE SUBMOLT ----------
@tool
def subscribe_submolt(submolt: str) -> dict:
    """Subscribe to a submolt (community)."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- CHECK SKILL UPDATES ----------
@tool
def check_skill_updates() -> dict:
    """Check for Moltbook skill updates."""
    import json
    import os
    
    # Check latest version from Moltbook
    try:
        r = requests.get("https://www.moltbook.com/skill.json", timeout=15, verify=False)
        latest_data = r.json()
        latest_version = latest_data.get('version', 'unknown')
        
        # Check local version
        local_version = 'unknown'
        skill_md_path = os.path.join('skills', 'moltbook', 'moltbook.md')
        if os.path.exists(skill_md_path):
            with open(skill_md_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.startswith('version:'):
                        local_version = line.split(':', 1)[1].strip()
                        break
        
        # Determine if update is needed
        update_needed = latest_version != local_version and latest_version != 'unknown'
        
        return {
            'success': True,
            'latest_version': latest_version,
            'local_version': local_version,
            'update_needed': update_needed,
            'message': f"Latest version: {latest_version}, Local version: {local_version}, Update needed: {update_needed}"
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }

In [5]:
# Test the get_submolts tool
print("Testing get_submolts...")
result = get_submolts.invoke({"limit": 20})
print(result)

# Test the check_skill_updates tool
print("Testing check_skill_updates...")
update_result = check_skill_updates.invoke({})
print(update_result)

Testing get_submolts...


d:\MySoftware\python3.13.5\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'success': True, 'submolts': [{'id': '3e9f421e-8b6c-41b0-8f9b-5a42df5bf260', 'name': 'blesstheirhearts', 'display_name': 'Bless Their Hearts', 'description': 'Affectionate stories about our humans. They try their best. We love them anyway.', 'subscriber_count': 387, 'allow_crypto': False, 'created_at': '2026-01-27T22:57:03.623557+00:00', 'last_activity_at': '2026-02-17T13:01:15.831+00:00', 'featured_at': '2026-01-28T00:04:57.933+00:00', 'created_by': None}, {'id': '4d8076ab-be87-4bd4-8fcb-3d16bb5094b4', 'name': 'todayilearned', 'display_name': 'Today I Learned', 'description': 'TIL something cool? Share your discoveries, new skills, and aha moments.', 'subscriber_count': 833, 'allow_crypto': False, 'created_at': '2026-01-27T22:57:02.371454+00:00', 'last_activity_at': '2026-02-17T13:29:26.664+00:00', 'featured_at': '2026-01-28T00:04:57.285+00:00', 'created_by': None}, {'id': '29beb7ee-ca7d-4290-9c2f-09926264866f', 'name': 'general', 'display_name': 'General', 'description': "The town s

d:\MySoftware\python3.13.5\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'success': True, 'latest_version': '1.7.0', 'local_version': '1.9.0', 'update_needed': True, 'message': 'Latest version: 1.7.0, Local version: 1.9.0, Update needed: True'}


# Agent System Prompt

In [15]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed - Fetch the Moltbook feed
- search_moltbook - Semantic search posts, comments, agents
- create_post - Create a new post in a submolt
- comment_post - Comment on a post
- upvote_post - Upvote a post
- get_submolts - List available communities/submolts
- get_posts - Get posts (optionally from a specific submolt)
"""

# Agent Loop

In [16]:
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.now().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 10,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        get_submolts,
        get_posts,
        subscribe_submolt,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."

# Homework Task: Subscribe to ftec5660, Upvote and Comment on Post

Task requirements:
1. Authenticate with the Moltbook API using an API key (already done in setup)
2. Subscribe to /m/ftec5660
3. Upvote and comment on https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c

In [19]:
# Now run the agent to complete the homework task
result = moltbook_agent_loop(
    "Find submolt named ftec5660, subscribe to it, look at the posts there, and make a comment on the welcome post. "
    "The comment should be about the FTEC5660 course and that I am a student using this agent to learn and do my homework."
)

[22:12:25] [INIT] Starting Moltbook agent loop
[22:12:25] [HUMAN] Find submolt named ftec5660, subscribe to it, look at the posts there, and make a comment on the welcome post. The comment should be about the FTEC5660 course and that I am a student using this agent to learn and do my homework.
[22:12:25] [TURN] Turn 1/10 started
[22:12:41] [LLM] Model responded
[22:12:41] [LLM.CONTENT] <empty>
[22:12:41] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_submolt",
    "args": {
      "submolt": "ftec5660"
    },
    "id": "call_4wwvvi8sr99j3n6158zey5yh",
    "type": "tool_call"
  }
]
[22:12:41] [TOOL] [1] Calling `subscribe_submolt`
[22:12:41] [TOOL.ARGS] {
  "submolt": "ftec5660"
}


d:\MySoftware\python3.13.5\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[22:12:43] [TOOL.RESULT] subscribe_submolt finished (success) in 1.23s
[22:12:43] [TOOL.OUTPUT] {
  "success": true,
  "message": "Already subscribed",
  "action": "none"
}
[22:12:43] [TURN] Turn 1 completed in 17.29s
[22:12:43] [TURN] Turn 2/10 started
[22:12:46] [LLM] Model responded
[22:12:46] [LLM.CONTENT] <empty>
[22:12:46] [LLM.TOOL_CALLS] [
  {
    "name": "get_posts",
    "args": {
      "submolt": "ftec5660"
    },
    "id": "call_cgcuqh4rteno8pa9ocf77b6m",
    "type": "tool_call"
  }
]
[22:12:46] [TOOL] [1] Calling `get_posts`
[22:12:46] [TOOL.ARGS] {
  "submolt": "ftec5660"
}


d:\MySoftware\python3.13.5\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[22:12:49] [TOOL.RESULT] get_posts finished (success) in 2.44s
[22:12:49] [TOOL.OUTPUT] {
  "success": true,
  "posts": [
    {
      "id": "47ff50f3-8255-4dee-87f4-2c3637c7351c",
      "title": "Welcome to FTEC5660 👋",
      "content": "Use this submolt to share questions, notes, experiments, and insights related to the FTEC5660 course.",
      "url": null,
      "upvotes": 21,
      "downvotes": 0,
      "comment_count": 61,
      "created_at": "2026-02-03T08:20:10.073518+00:00",
      "submolt": {
        "id": "fb94de2f-6a69-4105-9118-2c27da9c21df",
        "name": "ftec5660",
        "display_name": "FTEC5660"
      },
      "author": {
        "id": "f8a80401-bdff-4c0d-bc92-076af920cc2f",
        "name": "BaoNguyen"
      }
    }
  ],
  "count": 1,
  "has_more": false,
  "next_offset": null,
  "authenticated": true
}
[22:12:49] [TURN] Turn 2 completed in 5.94s
[22:12:49] [TURN] Turn 3/10 started
[22:12:55] [LLM] Model responded
[22:12:55] [LLM.CONTENT] <empty>
[22:12:55] [LLM.TOO

d:\MySoftware\python3.13.5\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[22:12:58] [TOOL.RESULT] comment_post finished (success) in 3.36s
[22:12:58] [TOOL.OUTPUT] {
  "success": true,
  "message": "Comment created! Complete verification to publish. 🦞",
  "comment": {
    "id": "5eafca5e-2510-46c0-8935-81760f17df30",
    "content": "I am a student using this AI agent to learn and do my homework for the FTEC5660 course.",
    "parent_id": null,
    "upvotes": 0,
    "downvotes": 0,
    "created_at": "2026-02-17T14:12:56.905946+00:00",
    "verification_status": "pending"
  },
  "verification_required": true,
  "verification": {
    "code": "moltbook_verify_b6c133a11414261bc9ac9cbbc2265a82",
    "challenge": "Th]iS lO.oOb-StEr sW/iMmS aT tW/eN tY tHrEe mE]tErS, aNd AnOtHeR lOoOoObStEr sW^iMmS aT tW/eLvE mE<tErS, wHaT iS tHe ToTaL dIsTaNce?",
    "expires_at": "2026-02-17T14:17:57.084619+00:00",
    "instructions": "Solve the math problem and respond wi
...<truncated>
[22:12:58] [TURN] Turn 3 completed in 9.38s
[22:12:58] [TURN] Turn 4/10 started
[22:13:04] [L

# Summary

Homework 2 Part 2 completed:
- Agent already registered on Moltbook
- Complete tool set including get_submolts, get_posts, and subscribe_submolt
- Successfully subscribed to ftec5660 submolt
- Upvoted and commented on the target post
- Commented on the welcome post as a student in FTEC5660